<center>
<h1> Statistical Algorithms for Change Detection in Optical and SAR Imagery: </h1>

<h1> Implementations in Python and on the Google Earth Engine </h1>

<h2> Mort Canty, Jülich Germany </h2>
<h2> Allan A. Nielsen, DTU Denmark   </h2> 
<h2> </h2>

<h2> <em>GI Forum, Institute for Geoinformatics  (ifgi),  </em></h2>
<h2> <em>University of Münster, June 13, 2017 </em></h2>
</center>


## Polarimetric SAR Imagery

<a href = "http://www2.imm.dtu.dk/pubdb/views/publication_details.php?id=6825"> 
	K. Conradsen et al. (2016). Determining the points of
	change in time series of polarimetric SAR data. IEEE TGRS 54 (5) 3007-3024.</a>

### Scattering matrix

$$
\pmatrix{E_h^b \cr E_v^b}
=\pmatrix{S_{hh} & S_{hv}\cr S_{vh} & S_{vv}}\pmatrix{E_h^i \cr E_v^i}.
$$

### Vector representation of an SLC (single-look complex) image

$$
s = \pmatrix{S_{hh}\cr \sqrt{2}S_{hv}\cr S_{vv}},
$$

### Span image (inner product)

$$
{\rm span} = s^\top s = |S_{hh}|^2 + 2|S_{hv}|^2 + |S_{vv}|^2.
$$

### Polarimetric matrix representation (outer product)

$$
C = s s^\top = \pmatrix{ |S_{hh}|^2 & \sqrt{2}S_{hh}S_{hv}^* & S_{hh}S_{vv}^* \cr
                                     \sqrt{2}S_{hv}S_{hh}^* & 2|S_{hv}|^2 & \sqrt{2}S_{hv}S_{vv}^* \cr
                                     S_{vv}S_{hh}^* & \sqrt{2}S_{vv}S_{hv}^* & |S_{vv}|^2 }.
$$


### Multi-look image
#### quad pol
$$
\bar{C}  ={1\over n}\sum_{\nu=1}^n  s(\nu) s(\nu)^\top = \langle  s s^\top \rangle
 = \pmatrix{ \langle |S_{hh}|^2\rangle & \langle\sqrt{2}S_{hh}S_{hv}^*\rangle & \langle S_{hh}S_{vv}^*\rangle \cr
\langle\sqrt{2} S_{hv}S_{hh}^*\rangle & \langle 2|S_{hv}|^2\rangle & \langle\sqrt{2}S_{hv}S_{vv}^*\rangle \cr
\langle S_{vv}S_{hh}^*\rangle & \langle\sqrt{2}S_{vv}S_{hv}^*\rangle & \langle |S_{vv}|^2\rangle }
$$
#### dual pol
$$
\bar{C} = \pmatrix{ \langle |S_{hh}|^2\rangle & \langle S_{hh}S_{hv}^*\rangle \cr
\langle S_{hv}S_{hh}^*\rangle & \langle |S_{hv}|^2\rangle }
$$
#### single pol
$$
\bar{C} = \langle |S_{hh}|^2\rangle \quad 
$$



#### The $p\times p$ matrix

$$
X = n\bar{C}
$$

#### has the complex Wishart distribution with $n$ degrees of freedom

$$
p_{W_c}(X) ={|X|^{(n-p)}\exp(-{\rm tr}(\Sigma^{-1} X)) \over
\pi^{p(p-1)/2}|\Sigma|^{n}\prod_{i=1}^p\Gamma(n+1-i)},\quad n \ge p, \quad  (p = 1,2,3)
$$

#### Per-pixel change hypothesis test for two acquisitions:

$$
H_0: \Sigma_1 = \Sigma_2\quad H_1: \Sigma_1\ne \Sigma_2
$$

#### Likelihood ratio test statistic:

$$
{L_0(X_1,X_2,\hat\Sigma)\over L_1(X_1,X_2,\hat\Sigma_1,\hat\Sigma_2)} = Q \lt t => Change
$$

#### We get

$$
Q = \left\{ 2^{2p} \frac{ |X_1| |X_2|} {|X_1 + X_2|^2} \right\}^n
$$



#### For $k$ acquisitions, to test the  null hypothesis $H_0$
$$
H_0: \ \Sigma_1 = \Sigma_2 = \cdots = \Sigma_k
$$
#### against all alternatives (omnibus test), we use a likelihood ratio  test statistic
$$
Q = \left\{ k^{pk} \frac{\prod_{i=1}^k |X_i|}{|X_1+X_2+\dots +X_k|^{k}} \right\}^n
$$
#### In the limit of a large number of looks, $-2\log Q$ has a chi-square distribution with $(k-1)p²$ degrees of freedom under the null hypothesis.

#### The no-change probability (P-value) is then 
$$
P = {\rm Prob}(Q < q_{obs}) = {\rm Prob}(-2\log Q > -2\log q_{obs})
$$
#### or, in Python with $z = -2\log q_{obs}$,
$$
{\tt P =  1- np.chi2.cdf(z,[(k-1)*p**2])}
$$

#### We would typically reject the null hypothesis when $P < 0.01$


#### Given that
$$
\Sigma_1=\Sigma_2=\cdots=\Sigma_{j-1}
$$
#### then the likelihood ratio test statistic $R_j$ for testing the hypothesis
$$
H_{0,j}: \Sigma_j=\Sigma_1 \ \ {\rm\bf against} \ \ H_{1,j}: \Sigma_j \neq \Sigma_1
$$
is
$$
R_j= \left\{\frac{j^{jp}}{(j-1)^{(j-1)p}}
\frac{|X_1+\cdots+X_{j-1}|^{(j-1)} |X_j|}{|X_1+\cdots+X_j|^j} \right\}^n
$$
####  The $R_j$ constitute a factorization of $Q$
$$
Q = \prod_{j=2}^k R_j
$$
#### If $H_0$ is true, the $R_j$ are independent.

#### Now the no-change P-value is

$$
P = {\rm Prob}(R_j < r_{j,obs}) = {\rm Prob}(-2\log R_j > -2\log r_{j,obs})
$$

### Algorithm:

#### Test $\Sigma_1 = \Sigma_2$

#### If not rejected, test $\Sigma_1 = \Sigma_2 = \Sigma_3$

#### and so on. 

#### If rejected, say  $\Sigma_1 = \Sigma_2 = \dots = \Sigma_{j-1} \ne \Sigma_j$

#### then re-start, testing $\Sigma_j = \Sigma_{j+1}$ and continue.

### Implementation in Docker

http://mortcanty.github.io/SARDocker/

In [1]:
!docker start sar


sar


http://localhost:433

In [ ]:
!docker stop sar

### Implementation on the GEE Python API

https://github.com/mortcanty/earthengine

In [ ]:
%cd /home/mort/python/earthengine/src
!./app.py

http://localhost:5000/